# Handle Text

## Text Tokenization

### Download and Read the Verdict
Download the Verdict as a text file, then read the file content.

In [20]:
import urllib.request
from msilib import type_key

url = ("https://raw.githubusercontent.com/rasbt/"
       "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
       "the-verdict.txt")
file_path = "./the-verdict.txt"
urllib.request.urlretrieve(url, file_path)

with open(file_path, "r") as file:
    raw_text = file.read()
print(f"Total characters: {len(raw_text)}")
print(raw_text[:100])

Total characters: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no g


### Split the Text into Words Using Regex

In [21]:
import re
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(len(preprocessed))

4690


### Convert the Words to Unique IDs


In [22]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)
print(vocab_size)

vocab = {token:integer for integer,token in enumerate(all_words)}
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

1130
('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


### Simple Tokenizer
Implement a encode function to convert a text into a sequence of token IDs.

Implement a decode function to convert a sequence of token IDs back into a text.

In [23]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.vocab = vocab
        self.inverse_vocab = {integer:token for token, integer in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        return [self.vocab[token] for token in preprocessed]

    def decode(self, ids):
        text = ' '.join([self.inverse_vocab[id] for id in ids])
        text = re.sub(r'\s+([,.:;?_!"()\'])', r'\1', text)
        return text

tokenizerV1 = SimpleTokenizerV1(vocab)
text = """It's the last he painted, you know," Mrs. Gisburn said with pardonable pride."""""
ids = tokenizerV1.encode(text)
print(ids)
decoded_text = tokenizerV1.decode(ids)
print(decoded_text)

[56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]
It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


### Tokenizer V2
We want to handle the unknown words in the text.
We will add a special token `<|unk|>` to represent the unknown words.

And add a token `<|endoftext|>` to represent the end of the text, thus we can handle multiple texts that from different sources.

In [28]:
# add the special tokens to the vocab
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])
vocab = {token:integer for integer,token in enumerate(all_tokens)}

class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.vocab = vocab
        self.inverse_vocab = {integer:token for token, integer in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        return [self.vocab.get(token, self.vocab["<|unk|>"]) for token in preprocessed]

    def decode(self, ids):
        text = ' '.join([self.inverse_vocab[id] for id in ids])
        text = re.sub(r'\s+([,.:;?_!"()\'])', r'\1', text)
        return text

tokenizerV2 = SimpleTokenizerV2(vocab)
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
text = " <|endoftext|> ".join((text1, text2))
ids = tokenizerV2.encode(text)
print(ids)
decoded_text = tokenizerV2.decode(ids)
print(decoded_text)


[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]
<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.


### BPE Tokenizer
BPE could handle the unknown words in the text by split it into smaller tokens.

BPE merge the most frequent pairs of tokens into a new token to make the vocabulary.

In [35]:
import tiktoken
tokenizer = tiktoken.get_encoding("gpt2")
text = "Aiwerkn oker"
ids = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(ids)
for id in ids:
    print(id, tokenizer.decode([id]))
print(tokenizer.decode(ids))

[32, 14246, 9587, 77, 267, 6122]
32 A
14246 iw
9587 erk
77 n
267  o
6122 ker
Aiwerkn oker
